## Avaliando [Crash Course](https://mxnet.apache.org/versions/master/api/python/docs/tutorials/getting-started/crash-course/2-create-nn.html)


### Step2

In [1]:
import mxnet as mx
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

mx.__version__

'2.0.0'

## Step 2 - Create neural network
Uma camada com 3 entradas e 5 saidas, com função de ativação "RELU"


In [2]:
layer=nn.Dense(5,in_units=3,activation='relu')
layer

Dense(3 -> 5, Activation(relu))

In [3]:
layer.initialize()

[10:17:14] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


#### dez linhas com três parametros

In [4]:
x=np.random.uniform(-1,1,(10,3))
x

array([[-0.88657403,  0.9273255 , -0.45468742],
       [-0.23311698, -0.04466975,  0.5834501 ],
       [ 0.62433743,  0.0577898 , -0.04004568],
       [ 0.13608909, -0.21443039,  0.8511933 ],
       [ 0.6721575 , -0.8579279 , -0.32520765],
       [-0.8257414 ,  0.2963438 , -0.9595632 ],
       [-0.2635169 ,  0.6652397 ,  0.91431034],
       [ 0.5563135 , -0.7192985 ,  0.7400243 ],
       [ 0.74017453,  0.95723665, -0.05278391],
       [ 0.59831715,  0.60182154, -0.07704127]])

In [5]:
layer(x)

array([[0.        , 0.        , 0.04437108, 0.01096384, 0.        ],
       [0.01540359, 0.01735983, 0.        , 0.        , 0.03530429],
       [0.00381139, 0.02891595, 0.0071606 , 0.01262893, 0.        ],
       [0.02378628, 0.04612946, 0.        , 0.        , 0.05157538],
       [0.        , 0.00375629, 0.        , 0.        , 0.00060787],
       [0.        , 0.        , 0.01947428, 0.00730047, 0.        ],
       [0.03439108, 0.04268821, 0.02091249, 0.        , 0.03359453],
       [0.01674652, 0.05354869, 0.        , 0.        , 0.05611669],
       [0.01591048, 0.04680086, 0.05175146, 0.03321034, 0.        ],
       [0.00959048, 0.03363482, 0.03384075, 0.02388959, 0.        ]])

In [6]:
layer.params

{'weight': Parameter (shape=(5, 3), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### pesos e bias de entrada de cada elemento da camada interna de 5 neuronios.

In [7]:
layer.weight.data()

array([[ 0.0068339 ,  0.01299825,  0.0301265 ],
       [ 0.04819721,  0.01438687,  0.05011239],
       [ 0.00628365,  0.04861524, -0.01068833],
       [ 0.01729892,  0.02042518, -0.01618656],
       [-0.00873779, -0.02834515,  0.05484822]])

In [8]:
layer.bias.data()


array([0., 0., 0., 0., 0.])

Chain layers into a neural network using nn.Sequential

Sequential é a ligação de camadas onde a saída de uma camada é a entrada de outra

In [9]:
net = nn.Sequential()

net.add(nn.Dense(5,in_units=3,activation='relu'),nn.Dense(25,activation='relu'),nn.Dense(2))

net

Sequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [10]:
net[1]

Dense(-1 -> 25, Activation(relu))

#### Custom neural network architecture flexibly

In [11]:
class Net(nn.Block):
    def __init__(self):
        super().__init__()
        
    def forward(self,x):
        return x

In [12]:
class MLP(nn.Block):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(5,activation='relu')
        self.dense2=nn.Dense(25,activation='relu')
        self.dense3=nn.Dense(2)
        
    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

In [13]:
net=MLP()
net

MLP(
  (dense1): Dense(-1 -> 5, Activation(relu))
  (dense2): Dense(-1 -> 25, Activation(relu))
  (dense3): Dense(-1 -> 2, linear)
)

In [14]:
net.dense1.params

{'weight': Parameter (shape=(5, -1), dtype=float32),
 'bias': Parameter (shape=(5,), dtype=float32)}

#### Creating custom layers using Parameters

In [15]:
from mxnet.gluon import Parameter

weight=Parameter("custom_parameter_weight",shape=(5,-1))
bias=Parameter("custom_parameter_bias",shape=(5,-1))

weight,bias

(Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>),
 Parameter (shape=(5, -1), dtype=<class 'numpy.float32'>))

Custom layer com função linear sem função de ativação. 

w*x+b

In [16]:
class custom_layer(nn.Block):
    def __init__(self,out_units, in_units=0):
        super().__init__()
        
        # mxnet 2.0
        self.weight=Parameter("weight",shape=(in_units,out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
        # mxnet 1.8.0
        # self.weight = self.params.get('weight', shape=(in_units, out_units),allow_deferred_init=True)
        # self.bias = self.params.get('bias', shape=(out_units,),allow_deferred_init=True)

        
    def forward(self,x):
        return np.dot(x,self.weight.data())+self.bias.data()

In [17]:
dense=custom_layer(3,in_units=5)
dense.initialize()
dense(np.random.uniform(size=(4,5)))

array([[-0.02481367,  0.007271  ,  0.06815495],
       [-0.0675921 , -0.06589589,  0.04664002],
       [-0.05190407, -0.01203752,  0.05761838],
       [-0.03687233, -0.00421239,  0.06642268]])

### [LeNet](http://yann.lecun.com/exdb/lenet/)

In [18]:
class LeNet(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=nn.Dense(10)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet=LeNet()        

In [19]:
class LeNet_custom(nn.Block):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2D(channels=6,kernel_size=3,activation='relu')
        self.pool1=nn.MaxPool2D(pool_size=2,strides=2)
        self.conv2=nn.Conv2D(channels=16,kernel_size=3,activation='relu')
        self.pool2=nn.MaxPool2D(pool_size=2,strides=2)
        self.dense1=nn.Dense(120,activation='relu')
        self.dense2=nn.Dense(84,activation='relu')
        self.dense3=custom_layer(10,84)
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=self.pool2(x)
        x=self.dense1(x)
        x=self.dense2(x)
        x=self.dense3(x)
        return x

lenet_custom=LeNet_custom()        

In [20]:
image_data=np.random.uniform(-1,1,(1,1,28,28))

lenet.initialize()
lenet_custom.initialize()

print("LeNet:")
print(lenet(image_data))

print("Custom LeNet:")
print(lenet_custom(image_data))


LeNet:
[[ 0.00089368 -0.00399038  0.00011905  0.00163105 -0.00089482 -0.00210765
   0.00552988  0.00021146  0.00221306 -0.00193633]]
Custom LeNet:
[[ 0.02705444  0.06015968 -0.06345475  0.02835384  0.05397669 -0.0451556
   0.04637432 -0.00195986  0.00080239 -0.05106261]]


In [21]:
lenet.conv1.weight.data().shape, lenet.dense1.bias.data().shape,lenet_custom.conv1.weight.data().shape, lenet_custom.dense1.bias.data().shape


((6, 1, 3, 3), (120,), (6, 1, 3, 3), (120,))

#### Using predefined (pretrained) architectures

[Gluon CV model zoo](https://cv.gluon.ai/model_zoo/index.html)

[Gluon NLP model zoo](https://nlp.gluon.ai/model_zoo/index.html)

In [22]:
from mxnet.gluon import model_zoo

net=model_zoo.vision.resnet50_v2(pretrained=True)
net.hybridize()

dummy_input=np.ones(shape=(1,3,224,224))
output=net(dummy_input)
output.shape


(1, 1000)

### Deciding the paradigm for your network

In [23]:
net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add(nn.Dense(5,in_units=3,activation='relu'),
    nn.Dense(25,activation='relu'),nn.Dense(2))

net_hybrid_seq

HybridSequential(
  (0): Dense(3 -> 5, Activation(relu))
  (1): Dense(-1 -> 25, Activation(relu))
  (2): Dense(-1 -> 2, linear)
)

In [24]:
net_hybrid_seq.hybridize()

### Creating custom layers using Parameters(HbridBlocks API)

In [25]:
class CustomLayer(nn.HybridBlock):
    def __init__(self,out_units, in_units=-1):
        super().__init__()
        self.weight=Parameter("weight",shape=(in_units, out_units),allow_deferred_init=True)
        self.bias=Parameter("bias",shape=(out_units,),allow_deferred_init=True)
        
    def forward(self, x):
        print(self.weight.shape,self.bias.shape)
        return np.dot(x,self.weight.data())+self.bias.data()

    def infer_shape(self,x):
        print(self.weight.shape,x.shape)
        self.weight.shape=(x. shape[-1],self.weight.shape[1])


dense=CustomLayer(3)

dense.initialize()
dense(np.random.uniform(size=(4,5)))
    

(-1, 3) (4, 5)
(5, 3) (3,)


array([[-0.00016055, -0.09525858, -0.02461037],
       [ 0.00876564, -0.08444627,  0.02188295],
       [ 0.04412797, -0.0623582 ,  0.03239645],
       [-0.00077351, -0.04988677, -0.0135352 ]])

### Performance

In [26]:
from time import time

def benchmark(net,x):
    y=net(x)
    start=time()
    for i in range(1,1000):
        y=net(x)
    return time()-start


x_bench=np.random.normal(size=(1,521))

net_hybrid_seq=nn.HybridSequential()

net_hybrid_seq.add (nn.Dense(256,activation='relu'),
                nn.Dense(128,activation='relu'),
                nn.Dense(2))

net_hybrid_seq.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid_seq, x_bench)))
net_hybrid_seq.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid_seq, x_bench)))

Before hybridizing: 0.4124 sec
After hybridizing: 0.1314


In [27]:
from mxnet.gluon import HybridBlock

class MLP_Hybrid(HybridBlock):
    def __init__(self):
        super().__init__()
        self.dense1=nn.Dense(256,activation='relu')
        self.dense2=nn.Dense(123,activation='relu')
        self.dense3=nn.Dense(2)

    def forward(self,x):
        layer1=self.dense1(x)
        layer2=self.dense2(layer1)
        layer3=self.dense3(layer2)
        return layer3

net_hybrid=MLP_Hybrid()
net_hybrid.initialize()

print('Before hybridizing: %.4f sec'%(benchmark(net_hybrid, x_bench)))
net_hybrid.hybridize()
print('After hybridizing: %.4f'%(benchmark(net_hybrid, x_bench)))

Before hybridizing: 0.3998 sec
After hybridizing: 0.1177


#### Saving and Loading your models

In [28]:
file_name='../../../data/crashCourse/models/layer.params'
layer.save_parameters(file_name)

In [29]:
def build_model():
    layer=nn.Dense(5,in_units=3,activation='relu')
    return layer

layer_new=build_model()

In [30]:
layer_new.load_parameters(file_name)

#### Save/load the model weights/parameters and the architectures

In [31]:
net_hybrid.export('../../../data/crashCourse/models/MLP_hybrid')

('../../../data/crashCourse/models/MLP_hybrid-symbol.json',
 '../../../data/crashCourse/models/MLP_hybrid-0000.params')

In [32]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    net_loaded=nn.SymbolBlock.imports('../../../data/crashCourse/models/MLP_hybrid-symbol.json',['data'], '../../../data/crashCourse/models/MLP_hybrid-0000.params',device=None)

In [33]:
net_loaded(x_bench)

array([[ 0.11246412, -0.0175273 ]])

#### Visualizing Models

In [34]:
layer.summary(x)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                                     (10, 3)               0
        Activation-1                                     (10, 5)               0
             Dense-2                                     (10, 5)              20
Parameters in forward computation graph, duplicate included
   Total params: 20
   Trainable params: 20
   Non-trainable params: 0
Shared params in forward computation graph: 0
Unique parameters in model: 20
--------------------------------------------------------------------------------


In [35]:
lenet.summary(image_data)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                              (1, 1, 28, 28)               0
        Activation-1                              (1, 6, 26, 26)               0
            Conv2D-2                              (1, 6, 26, 26)              60
         MaxPool2D-3                              (1, 6, 13, 13)               0
        Activation-4                             (1, 16, 11, 11)               0
            Conv2D-5                             (1, 16, 11, 11)             880
         MaxPool2D-6                               (1, 16, 5, 5)               0
        Activation-7                                    (1, 120)               0
             Dense-8                                    (1, 120)           48120
        Activation-9                                     (1, 84)               0
            Dense-10        

In [36]:
net_hybrid_summary=MLP_Hybrid()

net_hybrid_summary.initialize()

net_hybrid_summary.summary(x_bench)

net_hybrid_summary.hybridize()

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                                    (1, 521)               0
        Activation-1                                    (1, 256)               0
             Dense-2                                    (1, 256)          133632
        Activation-3                                    (1, 123)               0
             Dense-4                                    (1, 123)           31611
             Dense-5                                      (1, 2)             248
        MLP_Hybrid-6                                      (1, 2)               0
Parameters in forward computation graph, duplicate included
   Total params: 165491
   Trainable params: 165491
   Non-trainable params: 0
Shared params in forward computation graph: 0
Unique parameters in model: 165491
---------------------------------------------------